In [24]:
# %pip uninstall protobuf
# %pip install protobuf==3.20.3 --default-timeout=100
# %pip install -U -q "google-generativeai>=0.8.3"
# %pip install -U -q "google-generativeai>=0.8.3" chromadb

In [1]:
import google.generativeai as genai
from IPython.display import HTML, Markdown, display
import google.generativeai as genai
from IPython.display import Markdown
from dotenv import load_dotenv
import os

C:\Users\MSI\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
load_dotenv() 
api_key = os.getenv("API_KEY")
secret = os.getenv("SECRET")

print(api_key, secret)

In [26]:
genai.configure(api_key=GOOGLE_API_KEY)

In [27]:
flash = genai.GenerativeModel('gemini-1.5-flash')
response = flash.generate_content("explain ml shortly")
print(response.text)

Machine learning (ML) is a type of artificial intelligence (AI) that allows software applications to become more accurate in predicting outcomes without being explicitly programmed to do so.  It does this by analyzing large amounts of data, identifying patterns, and learning from them to make predictions or decisions.  Think of it as teaching computers to learn from experience instead of relying solely on pre-programmed rules.



In [28]:
DOCUMENT1 = "Our stickers are designed with high-quality, durable materials that are both water-resistant and fade-resistant, making them perfect for indoor and outdoor use. We offer a wide variety of styles, shapes, and finishes, including matte, glossy, and transparent options. Whether you're looking for personalized designs, logos, or artwork, our team ensures vibrant colors and sharp details in every sticker. Plus, all our stickers are easy to apply and can be removed without leaving residue, perfect for temporary or permanent use on laptops, bottles, cars, and more!"
DOCUMENT2 = 'Creating your custom sticker is simple and flexible! You can upload your own design, artwork, or logo directly on our website, or choose from a range of templates and add personalized text and images. Our easy-to-use design tool allows you to preview and adjust the size, colors, and finish before placing your order. If you need any help with design adjustments or have specific customization requests, our support team is here to guide you every step of the way, ensuring your stickers turn out exactly as you envisioned.'
DOCUMENT3 = "Once you’ve finalized your design, our production team gets to work right away. Orders are typically processed within 3-5 business days, with options for expedited shipping if you need them sooner. We ship worldwide and offer free shipping on bulk orders! After placing your order, you'll receive tracking information to follow your stickers’ journey from our facility to your doorstep. If you have any questions about your order status, delivery time, or need assistance with anything else, just let us know—we’re here to help!"
DOCUMENT4 = "the price of dilevery is 5DT for Tunis, 3DT for sousse and 20DT everywhere else"

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3, DOCUMENT4]

In [29]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry


class GeminiEmbeddingFunction(EmbeddingFunction):
    document_mode = True

    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

        response = genai.embed_content(
            model="models/text-embedding-004",
            content=input,
            task_type=embedding_task,
            request_options=retry_policy,
        )
        return response["embedding"]

In [30]:
import chromadb

DB_NAME = "googlecardb"
embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

print(documents)

db.add(documents=documents, ids=[str(i) for i in range(len(documents))])

["Our stickers are designed with high-quality, durable materials that are both water-resistant and fade-resistant, making them perfect for indoor and outdoor use. We offer a wide variety of styles, shapes, and finishes, including matte, glossy, and transparent options. Whether you're looking for personalized designs, logos, or artwork, our team ensures vibrant colors and sharp details in every sticker. Plus, all our stickers are easy to apply and can be removed without leaving residue, perfect for temporary or permanent use on laptops, bottles, cars, and more!", 'Creating your custom sticker is simple and flexible! You can upload your own design, artwork, or logo directly on our website, or choose from a range of templates and add personalized text and images. Our easy-to-use design tool allows you to preview and adjust the size, colors, and finish before placing your order. If you need any help with design adjustments or have specific customization requests, our support team is here t

Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Insert of existing embedding ID: 0
Insert of existing embedding ID: 1
Insert of existing embedding ID: 2


In [31]:
embed_fn.document_mode = False

query = "how much does the dilevery cost for Kasserine?"

result = db.query(query_texts=[query], n_results=1)
[[passage]] = result["documents"]

Markdown(passage)

the price of dilevery is 5DT for Tunis, 3DT for sousse and 20DT everywhere else

In [32]:
passage_oneline = passage.replace("\n", " ")
query_oneline = query.replace("\n", " ")

prompt = f''' you are an ai assistant for a website where we sell stickers, the queries might be in arabic or english
respond with the same language as the question. you will recieve the question and the text in the database that contains the 
answer. formulate a precise short response, dont response to any question that is not related to the business of 
selling stickers.

QUESTION : {query_oneline}
PASSAGE : {passage_oneline}
'''

In [33]:
model = genai.GenerativeModel("gemini-1.5-flash-latest")
answer = model.generate_content(prompt)
Markdown(answer.text)

The delivery cost to Kasserine is 20DT.
